In [1]:
import psycopg2

In [2]:
conn = psycopg2.connect(host="localhost", dbname="testpython", user="vaibhavs10", password="")

In [3]:
c = conn.cursor()

In [4]:
c.execute("CREATE TABLE task (id INT PRIMARY KEY, dependent_id INT REFERENCES task);")

In [5]:
c.execute("INSERT INTO task VALUES (6,8), (3,5), (4,8), (5, NULL), (1,3), (8,3), (2,1), (7,4);")

In [6]:
c.execute("WITH RECURSIVE traverse AS (SELECT id FROM task WHERE dependent_id=3 UNION ALL SELECT task.id FROM task INNER JOIN traverse ON task.dependent_id = traverse.id) SELECT id FROM traverse;")

In [7]:
c.fetchall()

[(1,), (8,), (6,), (4,), (2,), (7,)]

In [8]:
c.execute("WITH RECURSIVE traverse AS (SELECT id FROM task WHERE dependent_id=1 UNION ALL SELECT task.id FROM task INNER JOIN traverse ON task.dependent_id = traverse.id) SELECT id FROM traverse;")

In [9]:
c.fetchall()

[(2,)]

In [12]:
c.execute("WITH RECURSIVE traverse AS (SELECT id FROM task WHERE dependent_id=1 OR dependent_id=8 UNION ALL SELECT task.id FROM task INNER JOIN traverse ON task.dependent_id = traverse.id) SELECT id FROM traverse;")

In [13]:
c.fetchall()

[(6,), (4,), (2,), (7,)]

In [15]:
import networkx as nx

In [16]:
G1 = nx.Graph()
G1.add_edges_from([("a", "b"), ("b", "c"), ("c", "d"), ("d", "e"), ("f", "g")])

In [17]:
sorted(nx.connected_components(G1), key = len, reverse = True)

[{'a', 'b', 'c', 'd', 'e'}, {'f', 'g'}]

## Figuring out Networkx

In [20]:
import pandas as pd
import numpy as np

In [21]:
r = np.random.RandomState(seed=5)
ints = r.random_integers(1, 10, size=(3,2))
a = ['A', 'B', 'C']
b = ['D', 'A', 'E']
df = pd.DataFrame(ints, columns=['weight', 'cost'])
df[0] = a
df['b'] = b

/home/vaibhav/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: DeprecationWarning: This function is deprecated. Please call randint(1, 10 + 1) instead
  from ipykernel import kernelapp as app


In [22]:
df

,weight,cost,0,b
0,4,7,A,D
1,7,1,B,A
2,10,9,C,E


In [23]:
G = nx.from_pandas_dataframe(df, 0, 'b', ['weight', 'cost'])

In [35]:
G['A']

{'B': {'cost': 1, 'weight': 7}, 'D': {'cost': 7, 'weight': 4}}

The function call for nx.from_pandas_dataframe takes in the following parameters:

1. df
2. source - Column name for the source nodes
3. target - Column name for the target nodes
4. edge_attr - single/ list of column names 

In [33]:
for line in nx.generate_adjlist(G, delimiter=','):
    print line

A,B,D
C,E
B
E
D


In [36]:
nx.shortest_path(G)

{'A': {'A': ['A'], 'B': ['A', 'B'], 'D': ['A', 'D']},
 'B': {'A': ['B', 'A'], 'B': ['B'], 'D': ['B', 'A', 'D']},
 'C': {'C': ['C'], 'E': ['C', 'E']},
 'D': {'A': ['D', 'A'], 'B': ['D', 'A', 'B'], 'D': ['D']},
 'E': {'C': ['E', 'C'], 'E': ['E']}}

In [40]:
nx.shortest_path(G, source='A', target='D')

['A', 'D']

In [42]:
from networkx.algorithms.traversal.depth_first_search import dfs_tree

In [45]:
dfs_tree(G, 'B')

In [56]:
G.edges(['A'])

[('A', 'B'), ('A', 'D')]

# Generating test data to test all the functions

In [57]:
df = pd.DataFrame({'name': ['A', 'B', 'C', 'D', 'E'], 
                   'manager': ['Z', 'Z', 'Z', 'X', None],
                   'join_date': ['10/11/1995','10/11/1996','10/11/1997','10/11/1998','10/11/1999']})

In [80]:
df = pd.DataFrame({'name': ['A', 'B', 'C', 'D', 'E'], 
                   'manager': ['Z', 'Z', 'Z', 'X', None],
                   'join_date': ['10/11/1995','10/11/1996','10/11/1997','10/11/1998','10/11/1999'],
                   'assistant': ['S', 'Q', 'Q', 'Q', None],
                   'position': ['Executive', 'Executive', 'Freelancer', 'Manager', 'Manager']})

In [81]:
df

,assistant,join_date,manager,name,position
0,S,10/11/1995,Z,A,Executive
1,Q,10/11/1996,Z,B,Executive
2,Q,10/11/1997,Z,C,Freelancer
3,Q,10/11/1998,X,D,Manager
4,None,10/11/1999,None,E,Manager


In [87]:
G = nx.from_pandas_dataframe(df, 'manager', 'name', ['join_date', 'position', 'assistant'])

In [88]:
G['A']

{'Z': {'assistant': 'S', 'join_date': '10/11/1995', 'position': 'Executive'}}

In [89]:
G.degree()

1

In [92]:
nx.edges(G, 'A')

[('A', 'Z')]

# Finding shortest path b/w 2 nodes

In [67]:
nx.shortest_path(G, source='A', target='B')

['A', 'Z', 'B']

# Finding nodes in a subtree with a particular value

In [68]:
a = (n for n in G if G.node[n]['join_date'] == '10/11/1995')

In [69]:
a

<generator object <genexpr> at 0x7f6437301a50>

In [72]:
nodes = filter(lambda (n,d): d['join_date']=='10/11/1995', G.nodes(data=True))

KeyError: 'join_date'

In [79]:
G['A']['Z']

{'join_date': '10/11/1995'}

In [78]:
G.degree()

1

In [93]:
G.adjacency_list()

[['Z'], ['Z'], ['Z'], [None], ['X'], ['E'], ['D'], ['A', 'C', 'B']]

In [94]:
df = pd.DataFrame({'name': ['A', 'B', 'C', 'D', 'E'], 
                   'manager': ['Z', 'Z', 'Z', 'X', None],
                   'join_date': ['10/11/1995','10/11/1996','10/11/1997','10/11/1998','10/11/1999'],
                   'assistant': ['S', 'Q', 'Q', 'Q', None],
                   'position': ['Executive', 'Executive', 'Freelancer', 'Manager', 'Manager']})

In [95]:
G = nx.from_pandas_dataframe(df, 'manager', 'name', ['join_date', 'position', 'assistant'])

In [97]:
G['A']

{'Z': {'assistant': 'S', 'join_date': '10/11/1995', 'position': 'Executive'}}

# Map relations not variables

In [143]:
conn = psycopg2.connect(host="localhost", dbname="testpython", user="vaibhavs10", password="")
c = conn.cursor()

# Table structure for employees

1. employee_id is the id of the employeed
2. dependent_id is the id of the manager

In [106]:
c.execute("CREATE TABLE employees (employee_id INT PRIMARY KEY, dependent_id INT REFERENCES employees);")

In [107]:
conn.commit()

In [108]:
c.execute("INSERT INTO employees VALUES (1,2), (2,4), (4,5), (8, NULL), (6, 26), (26,4), (5,8)")

In [109]:
conn.commit()

# Table structure for employee_detail

1. employee_id
2. join_date

In [110]:
c.execute("CREATE TABLE employee_detail (employee_id INT PRIMARY KEY, join_date VARCHAR)")

In [111]:
conn.commit()

In [112]:
c.execute("INSERT INTO employee_detail VALUES (1, '10/11/2002'), (2, '10/11/2001'), (4, '10/11/2000'), (5, '10/11/1999'), (8, '10/11/1998'), (6, '10/11/2004'), (26, '10/11/2003') ")

In [113]:
conn.commit()

In [114]:
c.execute("SELECT * FROM employee_detail")

In [115]:
c.fetchall()

[(1, '10/11/2002'),
 (2, '10/11/2001'),
 (4, '10/11/2000'),
 (5, '10/11/1999'),
 (8, '10/11/1998'),
 (6, '10/11/2004'),
 (26, '10/11/2003')]

In [116]:
c.execute("SELECT * FROM employees")

In [117]:
c.fetchall()

[(1, 2), (2, 4), (4, 5), (8, None), (6, 26), (26, 4), (5, 8)]

In [123]:
c.execute("SELECT RECURSIVE traverse AS (\
          SELECT employee_id FROM employees\
          WHERE employee_id = 4 \
          UNION ALL \
          SELECT employee_id FROM employees \
          ON employees.employee_id = traverse.employee_id \
         ) SELECT employee_id FROM traverse;")

ProgrammingError: syntax error at or near "AS"
LINE 1: SELECT RECURSIVE traverse AS (          SELECT employee_id F...
                                  ^


### To get details of employee_id = 8

In [126]:
c.execute("SELECT * FROM employees WHERE employee_id = 8")
print c.fetchall()
c.execute("SELECT * FROM employee_detail WHERE employee_id = 8")
print c.fetchall()

[(8, None)]
[(8, '10/11/1998')]


# Query for Subtree

In [138]:
c.execute("WITH RECURSIVE traverse AS (\
SELECT employee_id FROM employees WHERE dependent_id = 4 \
UNION ALL \
SELECT employees.employee_id FROM employees \
INNER JOIN traverse \
ON employees.dependent_id = traverse.employee_id) \
SELECT employee_id FROM traverse;")

In [139]:
c.fetchall()

[(2,), (26,), (1,), (6,)]

# Query to get the whole tree by node number

In [144]:
c.execute("WITH RECURSIVE traverse AS (\
SELECT employee_id FROM employees WHERE dependent_id = 4 \
UNION ALL \
SELECT employees.employee_id FROM employees ) \
SELECT employee_id FROM traverse;")

In [145]:
c.fetchall()

[(2,), (26,), (1,), (2,), (4,), (8,), (6,), (26,), (5,)]

In [148]:
c.execute("WITH RECURSIVE traverse(employee_id, depth) \
AS ( \
SELECT employee_id, 0 FROM employees \
WHERE dependent_id = 4 \
UNION ALL \
SELECT employees.employee_id, traverse.depth + 1 FROM employees \
INNER JOIN traverse ON employees.dependent_id = traverse.employee_id) \
SELECT employee_id, depth FROM traverse ORDER BY depth DESC;")

In [149]:
c.fetchall()

[(1, 1), (6, 1), (2, 0), (26, 0)]

# JSON Format

{relations: [(1, 2), (2, 3), (3, 4)],

values: [(1, "22/11/2014"), (2, "30/02/2016"), (3, "12/04/1993"), (4, "10/11/1995")]}

In [150]:
c.execute("SELECT * FROM employees;")
data = c.fetchall()

In [152]:
type(data)

list

In [153]:
G = nx.Graph()
G.add_edges_from(data)

In [154]:
G

In [156]:
nx.shortest_path(G, 1, 8)

[1, 2, 4, 5, 8]